In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import clustering_analysis
from clustering_analysis import Cluster
import numpy as np
import seaborn as sns
import scipy.stats
import scipy.spatial
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

matplotlib.rcParams['font.family'] = "Inter"
matplotlib.rcParams["savefig.bbox"] = 'tight'

In [ ]:
def get_partial_eta_squared(lm):
    if not isinstance(lm, statsmodels.regression.linear_model.RegressionResultsWrapper):
        raise Exception('Invalid argument, need regression model')

    aov = statsmodels.stats.anova.anova_lm(lm)
    
    sseffect = aov["sum_sq"].iloc[:-1]

    sstotal = np.sum(aov["sum_sq"])

    partial_eta_squared = sseffect / sstotal
    partial_eta_squared.name = 'partial_eta_squared'

    return partial_eta_squared

GRAPH_LABELS = {
    "area": "Area ($\\mathrm{px}^2$)",
    "numerosity": "Cluster numerosity",
    "convex_hull_point_percentage": "Convex hull point percentage (%)",
    "density": "Density (points per $\\mathrm{px}^2$)",
    "linearity": "Linearity ($R^2$)",
    "cluster_structure": "Cluster structure",
    "number_of_points": "Number of points",
    "duration": "Duration (ms)",
    "n_clusters": "Number of clusters",
    "contribution": "Cluster contribution",
    "clustered": "Clustered",
    "disperse": "Dispersed",
    "dispersed": "Dispersed"
}


In [ ]:
clusters_df = clustering_analysis.get_clusters_dataframe()
clusters_df['contribution'] = clusters_df['numerosity'] / clusters_df['number_of_points'] * 100
clusters_df.head()

In [ ]:
tmp = clusters_df.groupby(['participant_id', 'trial_number', 'number_of_points', 'cluster_structure']).agg({'start_time': 'min'}).reset_index()
print("Median start time", np.median(tmp['start_time']))
tmp

In [ ]:
sns.barplot(x="number_of_points", y="start_time", hue='cluster_structure', data=tmp.groupby(['number_of_points', 'cluster_structure']).agg({'start_time': 'median'}))

In [ ]:
# Normality tests

print(clusters_df[['cluster_normality_x', 'cluster_normality_y']].dropna().describe())

tmp = clusters_df['cluster_normality_y']

print(len(tmp[tmp < 0.05]) / len(tmp))

In [ ]:
fig, ax = plt.subplots()
ax.hist(clusters_df['cluster_normality_x'], bins=30)
ax.axvline(0.05, color='black')

In [ ]:
# 112 trials per participant

tmp = clusters_df.groupby(['participant_id', 'trial_number']).count().reset_index()[['participant_id', 'trial_number']].groupby('participant_id').count().reset_index()['trial_number']

assert len(np.unique(tmp)) == 1
assert tmp.iloc[0] == 112

In [ ]:
# Individual differences in the number of clusters

tmp = clusters_df.groupby('participant_id').agg(number_of_clusters=('duration', 'count')).reset_index()

tmp['number_of_clusters'].hist(bins=20)

## Number of clusters analyses

In [ ]:
tmp0 = clusters_df.groupby(['participant_id', 'number_of_points', 'cluster_structure', 'trial_number']).agg(n_clusters=('numerosity', 'count')).reset_index()
tmp1 = tmp0.groupby(['cluster_structure', 'number_of_points']).agg(dict(n_clusters='mean')).reset_index()

tmp1['cluster_structure'] = tmp1['cluster_structure'].map(GRAPH_LABELS)

fig, ax = plt.subplots(figsize=(4, 2))
sns.lineplot(x="number_of_points", y="n_clusters", hue="cluster_structure", data=tmp1, ax=ax)

ax.set_xlabel("Number of points")
ax.set_ylabel("Number of clusters")
ax.legend(title="Cluster Structure")
plt.savefig("number-of-clusters-by-number-of-points.png", dpi=600)

tmp2 = tmp0.groupby(['number_of_points']).agg(dict(n_clusters='median')).reset_index()
print(tmp2)
print(tmp0['n_clusters'].std())

In [ ]:
tmp = clusters_df.groupby(['participant_id', 'number_of_points', 'cluster_structure', 'trial_number']).agg(n_clusters=('numerosity', 'count')).reset_index()

tmp = tmp.groupby(['participant_id', 'number_of_points', 'cluster_structure']).agg(dict(n_clusters='mean')).reset_index()

model = smf.ols("n_clusters ~ 1 + number_of_points * cluster_structure", data=tmp).fit()

print(model.summary())
print(get_partial_eta_squared(model))

## Numerosity investigations

In [ ]:
clusters_df["numerosity"].hist(bins=len(np.unique(clusters_df["numerosity"])))
ax = plt.gca()
ax.set_ylabel('Counts')
ax.set_xlabel('Numerosity')

In [ ]:
#
# Numerosities aggregated by participant, and then graphed
#

tmp = clusters_df.groupby('participant_id').agg({'numerosity': 'median'}).sort_values(['numerosity'])

tmp_vs = []
for number in np.unique(tmp['numerosity']):
    tmp_vs.append([number, len(tmp[tmp['numerosity'] == number]) / len(tmp)])

tmp_vs = np.array(tmp_vs)

# plt.plot(tmp_vs[:, 0], tmp_vs[:, 1])
tmp['numerosity'].hist()
# tmp['numerosity'].median()
tmp_vs


In [ ]:
# Numerosity coverage analysis

tmp = clusters_df.groupby(['numerosity']).agg(count=('area', 'count')).sort_values(['numerosity']).reset_index()

tmp['coverage'] = tmp['numerosity'] * tmp['count']

tmp = tmp.sort_values('coverage', ascending=False)

tmp["coverage_percentage"] = tmp["coverage"] / np.sum(tmp["coverage"])  * 100
tmp = tmp[:10]

sns.barplot(x=[str(x) for x in tmp['numerosity']], y=tmp["coverage_percentage"])
plt.savefig('coverage_percentage.png', dpi=600)
tmp

clusters_df.groupby(['participant_id', 'trial_number']).agg({'number_of_points': 'first'}).reset_index()['number_of_points'].mean()

In [ ]:
plt.scatter(clusters_df['numerosity'], clusters_df['area'])
plt.xlabel('Numerosity')
plt.ylabel('Area')

In [ ]:
plt.figure()
plt.ylabel('Convex hull point percentage')
plt.xlabel('Numerosity')
plt.scatter(clusters_df['numerosity'], clusters_df['convex_hull_point_percentage'])

In [ ]:
sns.barplot(x="number_of_points", y="numerosity", data=clusters_df)

## Area investigations

In [ ]:
tmp = clusters_df.groupby(['participant_id', 'number_of_points', 'cluster_structure']).agg({'area': 'median'}).reset_index()
# sns.barplot(x="cluster_structure", y="area", data=tmp)
tmp['area'].median()

In [ ]:
sns.barplot(x="number_of_points", y="area", data=tmp)

In [ ]:
tmp = clusters_df.groupby(['participant_id', 'number_of_points', 'cluster_structure'])

In [ ]:
tmp = clusters_df[clusters_df["numerosity"] > 2]
tmp["area"].describe()
tmp.groupby('participant_id').agg({'area': 'median'})['area'].median()


In [ ]:
np.sqrt(clusters_df["area"].median())

First, let's look at the distributions of the various attributes of the clusters

In [ ]:
def distribution_histograms(filter_proc=lambda x: x):
    
    fig, axs = plt.subplots(nrows=len(clustering_analysis.Cluster.attributes), figsize=(7, 10))
    fig.tight_layout()
    for idx, (ax, attribute) in enumerate(zip(axs, clustering_analysis.Cluster.attributes)):
        vector = filter_proc(clusters_df[attribute]) 
        ax.hist(vector, bins=20)
        ax.set_title(f"Distribution of values for {attribute}")
        ax.set_ylabel("Counts")
        ax.set_xlabel(attribute)

    fig.subplots_adjust(hspace=1)


# distribution_histograms()

As we see, there are long tails. Let us now graph the 75% percentile (only meaningful for left leaning histograms)

In [ ]:
def filter_proc(v):
    return v[v < np.percentile(v, 75)]

# distribution_histograms(filter_proc)

What are the median and mean values for the attributes?



In [ ]:
clusters_df[Cluster.attributes].describe()

Number of clusters with numerosity >= 8.

In [ ]:
len(clusters_df[clusters_df['numerosity'] > 8]) / len(clusters_df)

Are these variables normal?

In [ ]:
scipy.stats.normaltest(clusters_df[Cluster.attributes])

Nope...

Let's look at the correlation matrix between the various attributes

In [ ]:
tmp = clusters_df.groupby(['participant_id', 'trial_number']).agg(n_clusters=('numerosity', 'count')).reset_index()

clusters_df_with_n_clusters = clusters_df.merge(tmp, on=['participant_id', 'trial_number'])
# clusters_df_with_n_clusters['cluster_structure'] = np.where(clusters_df_with_n_clusters['cluster_structure'] == 'clustered', 1, 0)


In [ ]:

# I'm not including 'number_of_points' and 'cluster_structure' here
# because the data here aren't aggregrated, so it presents a
# misleading relationship with them

corr_df = clusters_df_with_n_clusters[Cluster.attributes + ['duration', 'n_clusters']].corr()


fig, ax = plt.subplots(figsize=(5.5, 5.5))

ax.matshow(corr_df, cmap=matplotlib.colormaps['RdBu'])

ax.set_xticks(list(range(len(corr_df.columns))))
ax.set_xticklabels(list(map(lambda x: GRAPH_LABELS[x], corr_df.columns)), rotation=90)
ax.set_yticks(list(range(len(corr_df.columns))))
ax.set_yticklabels(map(lambda x: GRAPH_LABELS[x], list(corr_df.columns)))

for i, cat in enumerate(corr_df.columns):
    for j, cat2 in enumerate(corr_df.columns):
        ax.text(i, j, round(corr_df.iloc[i, j], 2), ha='center', va='center', bbox={'facecolor': 'white', 'boxstyle': 'round', 'alpha': 0.9})

fig.savefig('attributes-correlation-matrix.png', dpi=600)

In [ ]:

# Latex Table Text
import scipy.stats

corr_df = clusters_df_with_n_clusters[Cluster.attributes + ['duration', 'n_clusters']].corr()
p_df = clusters_df_with_n_clusters[Cluster.attributes + ['duration', 'n_clusters']].corr(
    method=lambda x, y: scipy.stats.pearsonr(x, y)[1])

print("Variable & " + " & ".join(map(str, range(1, len(corr_df) + 1))) + " \\\\")

for idx, (rowvar, row) in enumerate(corr_df.iterrows()):
    print(f"{idx + 1}: " + GRAPH_LABELS[rowvar].replace("%", "\\%") + " & ", end='')
    for j in range(len(row)):
        p_val = p_df.iloc[idx, j]
        pstr = ''
        if p_val < 0.001:
            pstr = '***'
        elif p_val < 0.01:
            pstr = '**'
        elif p_val < 0.05:
            pstr = '*'
        # using pstr is pointless, everything is significant, less than 0.001
        print(str(round(corr_df.iloc[idx, j], 2)), end='')
        if j == len(row) - 1:
            pass
        else:
            print(" & ", end='')
    print(' \\\\')
    
# for i, cat in enumerate(corr_df.columns):
#     for j, cat2 in enumerate(corr_df.columns):
#         ax.text(i, j, round(corr_df.iloc[i, j], 2), ha='center', va='center', bbox={'facecolor': 'white', 'boxstyle': 'round', 'alpha': 0.9})



In [ ]:
p_df > 0.001

Now let's look at how the variables vary by number of points and cluster structure.

In [ ]:

def factor__heading(*args):
    text = " ".join([str(x) for x in args])
    print("")
    print("=" * len(text))
    print(text)
    print("=" * len(text))
    print("")

def factor_trends_for__graph_func(x, **kwargs):
    ax = plt.gca()
    ax.hist(x)

def factor_trends_for(data, attr, filter_proc=None, axs=None, stats=True):

    data = filter_proc(data) if filter_proc else data
    # fig = plt.figure()
    # g = sns.FacetGrid(col="number_of_points", row="cluster_structure", data=data, margin_titles=True)
    # g.map(factor_trends_for__graph_func, attr)
    # fig.suptitle(f"{attr} trends")

    if stats:

        factor__heading(f"Median {attr} (without aggregation by participant):", data[attr].median())
        factor__heading(f"Median {attr} (after aggregation by participant):", data.groupby(['participant_id']).agg({attr: 'median'})[attr].median())
        factor__heading(f"Mean {attr} (without aggregation by participant):", data[attr].mean())
        factor__heading(f"Mean {attr} (after aggregation by participant):", data.groupby(['participant_id']).agg({attr: 'mean'})[attr].mean())
        
        if attr in ('area', 'density'):
            factor__heading(f"Median {attr} (numerosity < 3 removal, without aggregation by participant):", data[data["numerosity"] > 2][attr].median())
            factor__heading(f"Median {attr} (numerosity < 3 removal, after aggregation by participant):", data[data["numerosity"] > 2].groupby(['participant_id', 'cluster_structure', 'number_of_points']).agg({attr: 'median'})[attr].median())
        

    analysis_df = (data.groupby(['participant_id', 'cluster_structure', 'number_of_points'])
                   .agg({attr: 'median', 'numerosity': 'median'}).reset_index())

    # Print out the descriptive for each cell
    
    if stats:
        factor__heading("Description:")
        print(data.groupby(['cluster_structure', 'number_of_points']).agg({attr: ['mean', 'std', 'median']}))

    formula = f"{attr} ~ 1 + number_of_points * cluster_structure"
    # if attr == "numerosity":
    #     formula = f"{attr} ~ 1 + number_of_points + cluster_structure"

    # formula = f"{attr} ~ 1 + numerosity"

    if stats:
        model = smf.ols(formula, data=analysis_df)
        result = model.fit()
        factor__heading("Model summary")
        print(result.summary())
        factor__heading("Partial eta squared")
        print(get_partial_eta_squared(result))

    

    analysis_df['cluster_structure'] = np.where(analysis_df['cluster_structure'] == "clustered", "Clustered", "Dispersed")

    tmp = analysis_df.groupby(['cluster_structure', 'number_of_points']).agg({attr: 'mean'}).reset_index()
    max_val = np.max(tmp[attr]) * 1.2

    if axs is None or axs is False:
      fig, axs = plt.subplots(1, 2, figsize=(5, 2))


    ax = axs[0]
    sns.barplot(x="number_of_points", y=attr, data=analysis_df, color="tab:red", ax=ax)
    ax.set_xlabel(GRAPH_LABELS["number_of_points"])
    ax.set_ylabel(GRAPH_LABELS[attr])
    ax.set_ylim([0, max_val])

    # plt.savefig(f"descriptive-number-of-points-{attr}.png", dpi=300)

    ax = axs[1]
    sns.barplot(x="cluster_structure", y=attr, data=analysis_df, ax=ax)
    ax.set_xlabel(GRAPH_LABELS["cluster_structure"])
    ax.set_ylabel(GRAPH_LABELS[attr])
    ax.set_ylim([0, max_val])
    
    # plt.savefig(f"descriptive-cluster-structure-{attr}.png", dpi=300)
    
    # for ax in axs:
    #     ax.label_outer()

    # fig.tight_layout()
    # plt.savefig(f"descriptive-number-of-points-cluster-structure-{attr}.png", dpi=600)
    
    

    # plt.figure(figsize=(2.5, 2))
    # sns.scatterplot(x="numerosity", y=attr, data=analysis_df)
    # ax = plt.gca()
    # ax.set_xlabel(GRAPH_LABELS["numerosity"])
    # ax.set_ylabel(GRAPH_LABELS[attr])
    # plt.savefig(f"descriptive-numerosity-{attr}.png", dpi=600)
    
    

    

    # model = smf.mixedlm(formula, groups="participant_id", re_formula="~ number_of_points + cluster_structure", data=analysis_df)
    # result = model.fit(method=['lbfgs'])
    # print(result.summary())

    
    

Cluster.attributes

In [ ]:
fig, all_axs = plt.subplots(nrows=2, ncols=len(Cluster.attributes), figsize=(15, 5), sharey=False)

for idx, attribute in enumerate(Cluster.attributes):
    axs = all_axs[:, idx]
    df = clusters_df
    if attribute in ('area', 'density', 'linearity'):
        df = df[df['numerosity'] > 2]
    factor_trends_for(clusters_df, attribute, axs=axs, stats=False)

fig.tight_layout()

fig.savefig('all-attribute-trends.png', dpi=600)
    


## Factor trends

### Numerosity

In [ ]:
len(clusters_df[clusters_df['numerosity'] == 1]) / len(clusters_df['numerosity'])

In [ ]:
len(clusters_df[clusters_df['numerosity'] == 2]) / len(clusters_df['numerosity'])

In [ ]:
len(clusters_df[clusters_df['numerosity'] == 3]) / len(clusters_df['numerosity'])

In [ ]:
len(clusters_df[clusters_df['numerosity'] <= 3]) / len(clusters_df['numerosity'])

In [ ]:
len(clusters_df[clusters_df['numerosity'] <= 2]) / len(clusters_df['numerosity'])

In [ ]:
factor_trends_for(clusters_df, "numerosity")


### Contribution

In [ ]:
factor_trends_for(clusters_df, "contribution")

## Area

In [ ]:
factor_trends_for(clusters_df, "area")

### Density

In [ ]:
factor_trends_for(clusters_df, "density", lambda x: x[x["numerosity"] > 2])

### Linearity

In [ ]:
factor_trends_for(clusters_df, "linearity", lambda x: x[x["numerosity"] > 2])

### Convex hull point percentage

In [ ]:
factor_trends_for(clusters_df, "convex_hull_point_percentage")

### Duration

In [ ]:
factor_trends_for(clusters_df, "duration")

In [ ]:
Cluster.attributes

## Attribute sequence analyses

In [ ]:
def get_per_trial_analysis_df(attribute):
    def _sequence_analyze(df):
        df = df.sort_values('cluster_index')
        
        max_index = np.argmax(df[attribute])
        min_index = np.argmin(df[attribute])
        
        max_value = df[attribute].iloc[max_index]
        min_value = df[attribute].iloc[min_index]

        

        max_first = False
        max_last = False
        min_first = False
        min_last = False
        mid_first = False
        mid_last = False

        last_cluster_index = df['cluster_index'].iloc[-1]
        first_cluster_index = df['cluster_index'].iloc[0]
        
        
        if max_index == last_cluster_index:
            max_last = True

        if min_index == last_cluster_index:
            min_last = True

        if max_index == first_cluster_index:
            max_first = True

        if min_index == first_cluster_index:
            min_first = True

        mid_first = not (max_first or min_first)
        mid_last = not (max_last or min_last)

        # Begin conservation

        if np.sum(max_value == df[attribute]) != 1:
            if max_first:
                mid_first = True
                max_first = False
            if max_last:
                mid_last = True
                max_last = False

        if np.sum(min_value == df[attribute]) != 1:
            if min_first:
                mid_first = True
                min_first = False
            if min_last:
                mid_last = True
                min_last = False


        assert (mid_first or max_first or min_first) == True

        ordinality = None

        if mid_first:
            ordinality = "mid_first"
        elif max_first:
            ordinality = "max_first"
        elif min_first:
            ordinality = "min_first"
        else:
            raise ImplementationError("Impossible!")
        
        slope, intercept, r, p, se = scipy.stats.linregress(df["cluster_index"], df[attribute])
        return pd.Series({'r2': r ** 2,
                          'r': r,
                          'attribute': attribute,
                          "max_first": max_first,
                          "min_first": min_first,
                          "mid_first": mid_first,
                          "mid_last": mid_last,
                          "max_last": max_last,
                          "min_last": min_last,
                          "attribute_ordinality": ordinality,
                          "n_clusters": len(df),
                          'base_uuid': df['base_uuid'].iloc[0],
                          "cluster_structure": df['cluster_structure'].iloc[0],
                          "number_of_points": df['number_of_points'].iloc[0]})
    
    
    return (clusters_df
            .groupby(['participant_id', 'trial_number']).filter(lambda x: len(x) > 2).reset_index()
            .groupby(['participant_id', 'trial_number'])
            .apply(_sequence_analyze)
            .reset_index())

def sequence_analyze(attribute, analysis):
    # remove perfectly determined items
    # analysis[analysis['r2'] != 1]
    fig, ax = plt.subplots()
    ax.hist(analysis["r2"], bins=30)
    ax.set_title(f"{attribute} R2s")
    ax.axvline(np.median(analysis["r2"]), c='black')
    ax.axvline(np.mean(analysis["r2"]), c='red')


In [ ]:
trial_analysis_dfs = {}
for attribute in Cluster.attributes:
    trial_analysis_dfs[attribute] = get_per_trial_analysis_df(attribute)
    # tmp = trial_analysis_dfs[attribute]

pd.concat([v for v in trial_analysis_dfs.values()]).to_csv("build/trial_attribute_analysis_df.csv")

In [ ]:
# Number of points and cluster_structure matter for max first

def tmp(what, variable):
    print(what, variable)
    for attr in Cluster.attributes:
        tmp = trial_analysis_dfs[attr]
        tmp = tmp.groupby(['participant_id', variable]).agg({what: 'mean'}).reset_index()
        model = smf.ols(f"{what} ~ {variable}", data=tmp).fit()
        print(" ", attr, "R^2 =", np.round(model.rsquared, 3), "p =", np.round(model.f_pvalue, 2))
    print('-----')

tmp('max_first', 'number_of_points')
tmp('min_first', 'number_of_points')

tmp('max_last', 'number_of_points')
tmp('min_last', 'number_of_points')

tmp('max_first', 'cluster_structure')
tmp('min_first', 'cluster_structure')

tmp('max_last', 'cluster_structure')
tmp('min_last', 'cluster_structure')



## Individual differences in linearity across participants

In [ ]:
fig, axs = plt.subplots(ncols=len(trial_analysis_dfs), figsize=(13, 2), sharey=True, sharex=True)

for idx, (key, value) in enumerate(trial_analysis_dfs.items()):
    ax = axs[idx]
    tmp1 = value[value['n_clusters'] > 2]
    tmp = tmp1.groupby(['participant_id']).agg({'r': 'mean'})
    # print('Median:', tmp['r2'].median())
    sns.histplot(x='r', data=tmp, ax=ax, color='tab:red')
    ax.set_xlabel(f"{GRAPH_LABELS[key]} r")
    ax.set_ylabel('# of participants')
    # ax.set_ylim(0, 18)
    # ax.set_xlim(-1, 1)

fig.tight_layout()
fig.savefig('attribute_sequentiality_r.png', dpi=600)

In [ ]:
fig, axs = plt.subplots(ncols=5, figsize=(13, 2), sharey=True, sharex=True)

for idx, (attribute, df) in enumerate(trial_analysis_dfs.items()):
    df = df[df['n_clusters'] > 2]
    ax = axs[idx]
    ax.hist(df['r2'], color='tab:red')
    
    ax.set_xlabel(f"{GRAPH_LABELS[attribute]} $R^2$s")

    ax.axvline(np.median(df['r2']), color='black')

    print(attribute, np.median(df['r2']))
    # sequence_analyze(attribute, df)

axs[0].set_ylabel('Count')

fig.tight_layout()

fig.savefig('attribute_sequentiality.png', dpi=600)



In [ ]:
def expected_and_observed_frequencies(df):
    number_of_trials = len(df)
    number_of_possible_mid_clusters = (df['n_clusters'] - 2).sum()
    expected_probabilities = [number_of_trials, number_of_possible_mid_clusters, number_of_trials] / np.sum([number_of_trials, number_of_possible_mid_clusters, number_of_trials])
    expected_frequencies = expected_probabilities * number_of_trials
    observed_frequencies = [len(df['max_first'][df['max_first'] == True]), len(df['mid_first'][df['mid_first'] == True]), len(df['min_first'][df['min_first'] == True])]
    result = scipy.stats.chisquare(observed_frequencies, expected_frequencies)
    return result


def attribute_first_chi_ps(df):
    chitest = expected_and_observed_frequencies(df)
    return pd.Series({'chisquared': chitest.statistic, "pvalue": chitest.pvalue, "attribute": df['attribute'].iloc[0]})
    
for attribute, df in trial_analysis_dfs.items():
    results = df.groupby("participant_id").apply(attribute_first_chi_ps)
    ppercentage = len(results[results['pvalue'] < 0.05]) / len(results)
    
    print(f"{attribute}: % of p < 0.05 = {ppercentage}")


In [ ]:

for attribute in Cluster.attributes:

    tmp = trial_analysis_dfs[attribute]
    number_of_possible_mid_clusters = (tmp['n_clusters'] - 2).sum()
    expected_probabilities = [len(tmp), number_of_possible_mid_clusters, len(tmp)] / np.sum([len(tmp), number_of_possible_mid_clusters, len(tmp)])
    expected_frequencies = expected_probabilities * len(tmp)
    observed_frequencies = [len(tmp['max_first'][tmp['max_first'] == True]), len(tmp['mid_first'][tmp['mid_first'] == True]), len(tmp['min_first'][tmp['min_first'] == True])]
    result = scipy.stats.chisquare(observed_frequencies, expected_frequencies)
    print(f"{attribute}: X^2 = {result.statistic}, p = {np.round(result.pvalue, 3)}\n\tObserved: {observed_frequencies}, Expected={np.round(expected_frequencies, 2)}, Total: {len(tmp)}")



In [ ]:
def first_last_analyze(attribute, analysis, ax):
    # things_to_plot = ['max_first', 'max_last', 'min_first', 'min_last', 'mid_first', 'mid_last']
    labels = {'max_first': 'Maximum', 'min_first': 'Mininum', 'mid_first': "Other"}
    things_to_plot = ['max_first', 'min_first', 'mid_first']
    bar_data = dict()
    for thing in things_to_plot:
        bar_data[thing] = len(analysis[analysis[thing] == True]) / len(analysis)

    print(bar_data)

    x = np.arange(len(bar_data))
    # ax.set_ylim(0, 1)
    ax.bar(x, bar_data.values(), color='tab:red')
    ax.set_title(f"{GRAPH_LABELS[attribute]}")
    ax.set_xticks(x, [labels[k] for k in bar_data.keys()])


fig, axs = plt.subplots(ncols=len(Cluster.attributes), figsize=(12, 2), sharey=True)
for idx, (attribute, df) in enumerate(trial_analysis_dfs.items()):
    first_last_analyze(attribute, df[df['n_clusters'] > 2], axs[idx])

fig.tight_layout()
fig.savefig('max_min_first.png', dpi=600)
    

In [ ]:


tmp = {}

METRICS =  ['max_first', 'max_last', 'min_first', 'min_last', 'mid_first', 'mid_last', 'r2']


for metric in METRICS:
    for idx, (attribute, analysis) in enumerate(trial_analysis_dfs.items()):
        analysis = analysis.sort_values(['participant_id', 'trial_number'])
        attribute_data = analysis[metric]
        col_index = (attribute, metric)
        tmp[col_index] = attribute_data
        

one_analysis = next(iter(trial_analysis_dfs.values()))
one_analysis = one_analysis.sort_values(by=['participant_id', 'trial_number'])


        
all_metric_info_df = pd.DataFrame(tmp)
all_metric_info_df['participant_id'] = one_analysis['participant_id']
all_metric_info_df['trial_number'] = one_analysis['trial_number']





In [ ]:
agg_dict = {}

for attribute in Cluster.attributes:
    for metric in METRICS:
        if "mid" not in metric:
            agg_dict[(attribute, metric)] = 'mean'
    
all_metric_means_df = all_metric_info_df.groupby(['participant_id']).agg(agg_dict)

all_metric_means_df.columns


In [ ]:
CLUSTER_SCORES = ['silhouette_score',
                  'calinski_harabasz_score',
                  'davies_bouldin_score']


CLUSTER_SCORES_LABELS = {'silhouette_score': "Silhouette coefficient",
                         'calinski_harabasz_score': "Calinski-Harabasz index",
                         'davies_bouldin_score': 'Davies-Bouldin index'}


agg_dict = {s: 'first' for s in CLUSTER_SCORES}
agg_dict["cluster_index"] = 'count'

trials_df = clusters_df.groupby(['participant_id', 'trial_number', 'number_of_points', 'cluster_structure']).agg(agg_dict).rename(columns={'cluster_index': 'n_clusters'}).reset_index()
tmp = all_metric_info_df.copy()
tmp.columns = ['_'.join(filter(lambda x: len(x) > 0, col)).strip() for col in tmp.columns.values]

trials_df = trials_df.merge(tmp, on=['participant_id', 'trial_number'])

trials_df

In [ ]:
tmp = trials_df.groupby(['number_of_points', 'cluster_structure'])[CLUSTER_SCORES].agg(['mean']).reset_index()
tmp_long = pd.melt(tmp, ['number_of_points', 'cluster_structure'], var_name='metric', value_name='score')
tmp_long['metric'] = tmp_long['metric'].map(CLUSTER_SCORES_LABELS)
tmp_long['cluster_structure'] = tmp_long['cluster_structure'].map(GRAPH_LABELS)

g = sns.FacetGrid(col='metric', hue='cluster_structure', data=tmp_long, sharey=False, margin_titles=True, despine=False)
g.set_titles(row_template="Metric = {row_name}", col_template="{col_name}")
g.map(sns.lineplot, "number_of_points", 'score')
g.set_ylabels('Score')
g.set_xlabels('Number of Points')
g.add_legend(title='Cluster Structure')
g.tight_layout()
g.figure.savefig('cluster_metrics.png', dpi=600)

In [ ]:
print(trials_df[CLUSTER_SCORES].corr().to_string())

In [ ]:
for cluster_score in CLUSTER_SCORES:
    df = trials_df.groupby('participant_id').agg({cluster_score: 'median', 'n_clusters': 'median'}).reset_index()
    fig, ax = plt.subplots()
    ax.hist(df[cluster_score], bins=20)
    #ax.set_title(f"Individual differences: {cluster_score}")
    #ax.set_xlabel("Participant")
    #ax.set_ylabel(cluster_score)
    fig, ax = plt.subplots()
    ax.scatter(df['n_clusters'], df[cluster_score])
    ax.set_title(f"Median number of clusters vs {cluster_score}")
    ax.set_xlabel('n_clusters')
    ax.set_ylabel(cluster_score)

In [ ]:
for cluster_score in CLUSTER_SCORES:
    fig, ax = plt.subplots()
    # remove extremities
    
    df = trials_df.dropna()
    df = df[df[cluster_score] < np.percentile(df[cluster_score], 75)]

    result = scipy.stats.linregress(df['n_clusters'], df[cluster_score])
    r2 = result.rvalue ** 2
    
    ax.scatter(df['n_clusters'], df[cluster_score])
    ax.set_title(f"# clusters vs {cluster_score}, R2 = {round(r2, 3)}")
    ax.set_xlabel("n_clusters")
    ax.set_ylabel(cluster_score)

In [ ]:
for attribute in Cluster.attributes:
    for metric in METRICS:
        col_name = attribute + "_" + metric
        for cluster_score in CLUSTER_SCORES:
            col = trials_df[col_name]
            score_vector = trials_df[cluster_score]
            if col.dtype == 'bool':
                sns.kdeplot(data=trials_df, x=cluster_score)
                # true_count = len(col[col == True])
                # false_count = len(col[col == False])
                # ax.bar()
                pass
            else:
                fig, ax = plt.subplots()
                ax.scatter(col, score_vector)
                ax.set_title(f"For {attribute}: scatterplot of {cluster_score} vs {metric}")


In [ ]:
bins = 30
cluster_score = 'davies_bouldin_score'

for attribute in Cluster.attributes:
    tmp = trials_df.dropna().sort_values([cluster_score])
    fig, ax = plt.subplots()
    for metric in METRICS:
        col_name = attribute + "_" + metric
        df = tmp.copy()
        df[cluster_score] = pd.qcut(df[cluster_score], bins, precision=2)
        df = df.groupby(cluster_score).agg({col_name: 'mean'}).reset_index()


        x = np.arange(len(df[cluster_score]))
        ax.plot(x, df[col_name], label=metric)

        ax.set_xticks(x, np.round(list(map(lambda x: x.mid, df[cluster_score])), 2))
        ax.xaxis.set_major_locator(plt.MaxNLocator(bins))
        ax.set_title(f"{attribute} metric percentages for {cluster_score}")
        fig.tight_layout()
        fig.legend()
    


In [ ]:
import sklearn.ensemble

df = trials_df.dropna()

X = df.iloc[:, 8:].to_numpy().astype('float32')
y = df["davies_bouldin_score"]

model = sklearn.ensemble.RandomForestRegressor()
model.fit(X, y)
print(model.score(X, y))

imps_df = pd.DataFrame(list(zip(df.columns[8:], model.feature_importances_)), columns=['metric', 'importance'])
imps_df.sort_values(['importance'], ascending=False)
    


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5)
scores

In [ ]:
score = 'davies_bouldin_score'
X = participant_factors_df.iloc[:,1:].to_numpy()
y = trials_df.dropna().groupby(['participant_id']).agg({score: 'median'})[score]
model = sm.OLS(y, X)

result = model.fit()

print(result.summary())

In [ ]:
model = smf.ols(data=trials_df, formula='n_clusters ~ 1 + number_of_points + cluster_structure')
result = model.fit()
result.summary()

In [ ]:

model = smf.mixedlm(data=trials_df, formula='n_clusters ~ 1 + number_of_points + cluster_structure', groups="participant_id", re_formula="~ cluster_structure")
result = model.fit()
result.summary()


In [ ]:
df = trials_df.dropna()

records = []
for col in df.columns[8:]:
    result = scipy.stats.pearsonr(df['davies_bouldin_score'], df[col])
    records.append(dict(measure=col, r2=np.round(result.statistic ** 2, 3), p=np.round(result.pvalue, 2)))

pd.DataFrame.from_records(records).sort_values('r2')

In [ ]:
df = trials_df.dropna()
metric = 'davies_bouldin_score'
tmp = df.groupby('participant_id').agg({metric: 'median', 'n_clusters': 'median'}).sort_values(metric)
scipy.stats.pearsonr(tmp[metric], tmp['n_clusters']).statistic ** 2

In [ ]:
tmp.sort_values('n_clusters', ascending=False)

In [ ]:
tmp = trials_df.groupby(['participant_id', 'number_of_points', 'cluster_structure']).agg({'n_clusters': 'median', 'silhouette_score': 'median'}).reset_index()
smf.ols("silhouette_score ~ 1 + number_of_points + cluster_structure", data=tmp).fit().summary()